In [ ]:
from transformers import pipeline
import torch

pipe = pipeline(
    "text-generation",
    model="tollefj/nordavind-7b-instruct-warm",
    torch_dtype=torch.float16,
    device="mps",  # mps if you're on an apple silicon mac, else "cuda" if nvidia-gpu or "cpu"
)

In [ ]:
system_prompt = 'Du er "Nordavind", en hjelpsom assistent.'

def make_prompt(inst, res=None):
    if not res:
        return f"""<s>{system_prompt} [INST] {inst} [/INST] \\n"""
    return f"""<s>{system_prompt} [INST] {inst} [/INST] \\n {res} </s>"""

def postprocess(output, first_sent=False):
    output = output.split("\\n")[-1].strip()
    # ignore hashtags as we often see #no_output
    output = output.split("#")[0].strip()
    # ignore incomplete sentences
    if not output.endswith("."):
        output = output.rsplit(".", 1)[0] + "."
    if first_sent:
        return output.split(".")[0] + "."
    return output

def generate(prompt, tokens=100, first_sent=False, sample=False, temperature=1.0):
    prompt = make_prompt(prompt)
    output = pipe(
        prompt,
        max_length=tokens,
        do_sample=sample,
        temperature=temperature,
        no_repeat_ngram_size=3,
    )
    output = output[0]["generated_text"]
    output = postprocess(output, first_sent=first_sent)
    print(output)


In [29]:
generate(
    "Hva er NTNU og IDI?",
    sample=True,
    tokens=100,
)

NTNU og Institutt for datateknologi og informatikk (IDI) er to forskjellige organisasjoner. NTNU er Norges største universitet, og består av syv fakulteter og 40 institutter. IDI er imidlertid bare en av de 40 enhetene som utgjør Fakultet for informasjonsteknologi og elektroteknikk.


In [30]:
generate("Forklar part-of-speech tagging, stemming/lemmatization og TF-IDF", tokens=200)

Part-of speech tagger er en prosess som brukes til å identifisere ordklasser i en tekst. Stemming/Lemmatizering er en teknikk som brukes for å redusere lengden på ord ved å fjerne suffikser og prefikser. TFIDF er en algoritme som brukes i naturlig språkbehandling for å beregne relevansen til et dokument basert på dets frekvens og invers dokumentfrekvens.  TF er et mål på hvor ofte et ord forekommer i et dokument. IDF er invers av dokumentfrekvensen, som er et tall som indikerer hvor viktig et ord er i et gitt dokument.  Til sammen gir disse tre teknikkene en måte å analysere tekst på og trekke ut relevant informasjon.


In [31]:
generate("Svart kort på følgende oppgve: 1. Briefly describe your understanding of POS tagging and its possible use-cases in context of text generation applications/language modeling.", tokens=150)

POS-tagger er en type språkmodell som brukes til å merke ord i tekst. Den bruker en forhåndsopplært modell for å identifisere ordklasser som substantiv, verb, adjektiv, preposisjoner, etc.
